In [ ]:
import cv2 as cv
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [41]:
from mtcnn.mtcnn import MTCNN

In [42]:
import os
import random

In [43]:
class FACELOADING:
  def __init__(self,directory):
    self.directory=directory
    self.target_size=(160,160)
    self.X=[]
    self.Y=[]
    self.detector=MTCNN()

  def extract_face(self,filename):
    img=cv.imread(filename)
    img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
    result=self.detector.detect_faces(img)
    x,y,w,h=result[0]['box']
    x,y=abs(x),abs(y)
    face=img[y:y+h,x:x+w]
    face_arr=cv.resize(face,self.target_size)
    return face_arr

  def load_faces(self,dir):
    FACES=[]
    for im_name in os.listdir(dir):
      try:
        path=dir+im_name
        single_face=self.extract_face(path)
        FACES.append(single_face)
      except Exception as e:
        pass
    return FACES

  def load_classes(self):
    for sub_dir in os.listdir(self.directory):
      path=self.directory+'/'+sub_dir+'/'
      FACES=self.load_faces(path)
      labels=[sub_dir for _ in range(len(FACES))]
      print(f'LOADED SUCCESSFULLY : {len(labels)}')
      self.X.extend(FACES)
      self.Y.extend(labels)
    return np.asarray(self.X), np.asarray(self.Y)

  def plot_images(self):
    for num,img in enumerate(self.X):
      ncols=3
      nrows=len(self.Y)//ncols+1
      plt.subplot(nrows,ncols,num+1)
      plt.imshow(img)
      plt.axis('off')


In [44]:
faceloading=FACELOADING('C:/Users/KIIT/Minor_Project/Cricket_dataset/Train')
X,Y=faceloading.load_classes()

1/1 [==============================] - 0s 53ms/step
LOADED SUCCESSFULLY : 8
1/1 [==============================] - 0s 35ms/step
LOADED SUCCESSFULLY : 11
1/1 [==============================] - 0s 38ms/step
LOADED SUCCESSFULLY : 12
1/1 [==============================] - 0s 37ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 42ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 41ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 36ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 57ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 53ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 53ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 37ms/step
LOADED SUCCESSFULLY : 8
1/1 [==============================] - 0s 68ms/step
LOADED SUCCESSFULLY : 10
1/1 [==============================] - 0s 37ms/step
LOADED SUCCESSFULLY : 10
1

In [39]:
temp=[]
attendence=[]
Y_list=Y.tolist()
temp=list(set(Y_list))
dict_sheet={'name':[]}
attendence_sheet=pd.DataFrame(dict_sheet)

attendence_sheet['name']=temp
attendence_sheet.insert(1,'Attendence','A')
# attendence_sheet.reset_index('name')
attendence_sheet
# print(attendence_sheet['name'].tolist())
99999999999999999999999999999999999999999999999999999999999/////////////////////////////////////////////'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''44444444444444444# print(attendence_sheet['Attendence'].tolist())

file_path=f'sections//section_1//section_1.csv'
path=os.path.join(os.getcwd(),file_path)
attendence_sheet1=pd.read_csv(path)
attendence_sheet1['ATTENDENCE']=attendence_sheet['Attendence']#.tolist()
attendence_sheet1
attendence_array=attendence_sheet1.values.tolist()
attendence_array
# attendence_sheet2=pd.DataFrame(attendence_matrix,columns =['NAME', 'ATTENDENCE'])

[['Arshadeep Singh', 2, 'A'],
 ['Bhuvaneshwar Kumar', 15, 'A'],
 ['Dinesh Kartik', 21, 'A'],
 ['Hardik Pandya', 33, 'A'],
 ['KL Rahul', 1, 'A'],
 ['Kuldeep Yadav', 23, 'A'],
 ['Mohammed Shami', 11, 'A'],
 ['MS Dhoni', 7, 'A'],
 ['Ravichandran Ashwin', 99, 'A'],
 ['Ravindra Jadeja', 8, 'A'],
 ['Rishabh Pant', 17, 'A'],
 ['Rohit Sharma', 45, 'A'],
 ['Shikhar Dhawan', 42, 'A'],
 ['Suryakumar Yadav', 63, 'A'],
 ['Virat Kohli', 18, 'A'],
 ['Yuzvendra Chahal', 3, 'A']]

In [45]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
  face_img=face_img.astype('float32')
  face_img=np.expand_dims(face_img,axis=0)
  yhat=embedder.embeddings(face_img)
  return yhat[0]

In [46]:
EMBEDDED_X=[]
for img in X:
 O EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X=np.asarray(EMBEDDED_X)


1/1 [==============================] - 0s 140ms/step


In [47]:
np.savez_compressed('C:/Users/KIIT/Minor_Project/sections/section_1/section_1.npz',EMBEDDED_X,Y)

In [9]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
encoder.fit(Y)
Y=encoder.transform(Y)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(EMBEDDED_X,Y,shuffle=True,random_state=17)

In [11]:
from sklearn.svm import SVC
model=SVC(kernel='linear',probability=True)
model.fit(X_train,Y_train)
ypreds_train=model.predict(X_train)
ypreds_test=model.predict(X_test)


In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_train,ypreds_train)
accuracy_score(Y_test,ypreds_test)


1.0

In [18]:
import joblib
joblib.dump(model,'pretrained_svm_face_detector.sav')

['pretrained_svm_face_detector.sav']

In [17]:
detector=MTCNN()
image_path='C:/Users/KIIT/Minor_Project/Cricket_dataset/Test/1.jpeg'
img=cv.imread(image_path)
img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
height,width,other=img.shape
height,width=height//2,width//2
print(height,width)
result=[]
output=[]
for _ in range(5):
    temp_y1,temp_x1 = random.randint(height-300, height+300),random.randint(width-300, width+300)
    img1 = img[0:temp_y1, 0:temp_x1]
    img2 = img[0:temp_y1, temp_x1 :len(img[0])]
    img3 = img[temp_y1:len(img), 0:temp_x1]
    img4 = img[temp_y1:len(img), temp_x1:len(img[0])]
    output1 = detector.detect_faces(img1)
    output2 = detector.detect_faces(img2)
    output3 = detector.detect_faces(img3)
    output4 = detector.detect_faces(img4)
    for item in output1:
        output.append(item['box'])
    # output=[item['box'] for item in output1]
    for item in output2:
        item['box'][0]+=temp_x1
        output.append(item['box'])
    for item in output3:
        item['box'][1]+=temp_y1
        output.append(item['box'])
    for item in output4:
        item['box'][0]+=temp_x1
        item['box'][1]+=temp_y1
        output.append(item['box'])
#     output.extend([*output1, *output2, *output3, *output4])
#     result=[item for item in output if item not in result]
# for item in output:
#     if item not in result:
#         result.append(item)
# result=list(set(output))
print(output)
present=[]
for item in output:
    x,y,w,h=item#['box']
    t_img=img[y:y+h,x:x+w]
#     print(t_img.shape)
#     plt.imshow(t_img)
    t_img=cv.resize(t_img,(160,160))
    test_img=get_embedding(t_img)
#     print(test_img)
    test_img=[test_img]
    ypreds=model.predict(test_img)
    print(ypreds)
    target=encoder.inverse_transform(ypreds)
    present.append(target[0])
present

400 600
2/2 [==============================] - 0s 18ms/step
[[421, 355, 36, 44], [129, 370, 35, 44], [284, 358, 34, 46], [154, 263, 36, 46], [282, 219, 34, 44], [569, 373, 36, 45], [1007, 222, 35, 45], [537, 245, 31, 41], [719, 361, 34, 46], [648, 257, 32, 42], [853, 362, 38, 47], [1010, 372, 35, 43], [285, 220, 31, 42], [284, 359, 34, 46], [154, 264, 34, 45], [131, 370, 32, 42], [854, 364, 37, 44], [538, 245, 31, 40], [718, 361, 35, 46], [568, 373, 37, 46], [421, 355, 35, 45], [1007, 223, 35, 44], [1010, 371, 36, 43], [648, 258, 32, 41], [537, 243, 32, 42], [283, 218, 34, 45], [155, 264, 36, 45], [1007, 222, 34, 44], [648, 257, 32, 42], [422, 360, 35, 40], [283, 361, 34, 45], [567, 372, 36, 48], [131, 370, 32, 42], [1013, 373, 33, 40], [720, 363, 33, 42], [855, 363, 34, 45], [651, 435, 19, 21], [421, 355, 35, 45], [537, 244, 31, 41], [154, 263, 36, 45], [283, 219, 34, 43], [283, 360, 34, 46], [131, 371, 33, 42], [569, 373, 29, 41], [854, 362, 37, 47], [1005, 221, 37, 46], [1010, 372, 

['k._l._rahul',
 'Bhuvi',
 'Dinesh_Kartik',
 'Rishab_Pant',
 'Arshadeep_singh',
 'rohit_sharma',
 'Ravichandran Ashwin',
 'Suryakumar_Yadav',
 'Hardik Pandya',
 'yuzvendra_chahal',
 'virat_kohli',
 'ravindra_jadeja',
 'Arshadeep_singh',
 'Dinesh_Kartik',
 'Rishab_Pant',
 'Bhuvi',
 'virat_kohli',
 'Suryakumar_Yadav',
 'Hardik Pandya',
 'rohit_sharma',
 'k._l._rahul',
 'Ravichandran Ashwin',
 'ravindra_jadeja',
 'yuzvendra_chahal',
 'Suryakumar_Yadav',
 'Arshadeep_singh',
 'Rishab_Pant',
 'Ravichandran Ashwin',
 'yuzvendra_chahal',
 'k._l._rahul',
 'Dinesh_Kartik',
 'rohit_sharma',
 'Bhuvi',
 'ravindra_jadeja',
 'Hardik Pandya',
 'virat_kohli',
 'Rishab_Pant',
 'k._l._rahul',
 'Suryakumar_Yadav',
 'Rishab_Pant',
 'Arshadeep_singh',
 'Dinesh_Kartik',
 'Bhuvi',
 'rohit_sharma',
 'virat_kohli',
 'Ravichandran Ashwin',
 'ravindra_jadeja',
 'yuzvendra_chahal',
 'Hardik Pandya',
 'Rishab_Pant',
 'Suryakumar_Yadav',
 'Arshadeep_singh',
 'Ravichandran Ashwin',
 'Suryakumar_Yadav',
 'yuzvendra_ch

In [21]:
print(len(present))

13


In [22]:
attendence_array=attendence_sheet.values.tolist()
attendence_array

[['mohammed_shami', 0, 'A'],
 ['shikhar_dhawan', 0, 'A'],
 ['Bhuvi', 0, 'A'],
 ['Ravichandran Ashwin', 0, 'A'],
 ['Arshadeep_singh', 0, 'A'],
 ['Dinesh_Kartik', 0, 'A'],
 ['kuldeep_yadav', 0, 'A'],
 ['Suryakumar_Yadav', 0, 'A'],
 ['virat_kohli', 0, 'A'],
 ['Hardik Pandya', 0, 'A'],
 ['ms_dhoni', 0, 'A'],
 ['ravindra_jadeja', 0, 'A'],
 ['yuzvendra_chahal', 0, 'A'],
 ['rohit_sharma', 0, 'A'],
 ['k._l._rahul', 0, 'A'],
 ['Rishab_Pant', 0, 'A']]

In [23]:
for student in attendence_array:
    if student[0] in present:
        student[1]=student[1]+1
        student[2]='P'
        
attendence_array

[['mohammed_shami', 1, 'P'],
 ['shikhar_dhawan', 0, 'A'],
 ['Bhuvi', 1, 'P'],
 ['Ravichandran Ashwin', 1, 'P'],
 ['Arshadeep_singh', 1, 'P'],
 ['Dinesh_Kartik', 1, 'P'],
 ['kuldeep_yadav', 0, 'A'],
 ['Suryakumar_Yadav', 1, 'P'],
 ['virat_kohli', 1, 'P'],
 ['Hardik Pandya', 1, 'P'],
 ['ms_dhoni', 0, 'A'],
 ['ravindra_jadeja', 1, 'P'],
 ['yuzvendra_chahal', 1, 'P'],
 ['rohit_sharma', 1, 'P'],
 ['k._l._rahul', 1, 'P'],
 ['Rishab_Pant', 1, 'P']]

In [24]:
attendence=pd.DataFrame(attendence_array,columns =['Name','Count', 'Attendence'])

In [25]:
attendence

,Name,Count,Attendence
0,mohammed_shami,1,P
1,shikhar_dhawan,0,A
2,Bhuvi,1,P
3,Ravichandran Ashwin,1,P
4,Arshadeep_singh,1,P
5,Dinesh_Kartik,1,P
6,kuldeep_yadav,0,A
7,Suryakumar_Yadav,1,P
8,virat_kohli,1,P
9,Hardik Pandya,1,P
